# About
- Build basic RAG chain
- Build RAG chain that also returns the sources

# Imports and Settings

In [17]:
import os
import gradio as gr

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv
load_dotenv()
OPENAI_APIKEY = os.environ['OPENAI_APIKEY']

# Create Basic RAG Chain

In [ ]:
# Instantiate embeddings model
embeddings_model = OpenAIEmbeddings(api_key=OPENAI_APIKEY, model='text-embedding-3-large', max_retries=100, chunk_size=16, show_progress_bar=False)

# Instantiate chat model
chat_model_4 = ChatOpenAI(api_key=OPENAI_APIKEY, temperature=0.5, model='gpt-4-turbo-2024-04-09')


In [ ]:
# Load chroma from disk
vectorstore = Chroma(persist_directory="../chroma_db/", embedding_function=embeddings_model)

# Set up the vectorstore to be the retriever
k = 5
retriever = vectorstore.as_retriever(search_kwargs={"k":k})

# Format docs function
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Test out the retriever

In [ ]:
# Test out the retriever
query = "eggs and diabetes"
results = retriever.invoke(query)

In [ ]:
# How would the formatted documents be rendered in the query?
print(format_docs(results))

Might be nice to also have the title!

In [ ]:
# Format docs function with the addition of the title
def format_docs(docs):
    return "\n\n".join((doc.metadata['title'] + '\n' + doc.page_content) for doc in docs)

In [ ]:
print(format_docs(results))

In [ ]:
# Get the prompt from the langchain hub. Or you could write your own!
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

## Test out the RAG_Chain

In [ ]:
# Create RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat_model_4
    | StrOutputParser())

In [ ]:
query = "eggs and diaetes"
rag_chain.invoke(query)

# Create simple gradio answer bot

In [ ]:
# Function to generate answer
def generate_answer(message, history):
    return rag_chain.invoke(message)

In [ ]:
# Set up chat bot interface
answer_bot = gr.ChatInterface(
                            generate_answer,
                            chatbot=gr.Chatbot(height=300),
                            textbox=gr.Textbox(placeholder="Ask me a question about nutrition and health", container=False, scale=7),
                            title="Nutrition Facts ChatBot",
                            description="Ask Dr Michael McGregor's Nutrition Facts videos any questions!",
                            theme="soft",
                            examples=["diverticulosis", "heart disease", "low carb diets", "diabetes", "green tea"],
                            cache_examples=False,
                            retry_btn=None,
                            undo_btn=None,
                            clear_btn=None,
                            submit_btn="Ask"
                        )

In [ ]:

answer_bot.launch()